In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.preprocessing import MinMaxScaler
from transformers import BertModel, BertTokenizer

## MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
import torch

def multinomialnb(vectorizer, data, input):
    if data == "fetch_20newsgroups":
        datasets = fetch_20newsgroups()
    x, y = datasets.data, datasets.target
    input = np.array([input])

    def document_to_vector(doc, model): # chuyển list token trong câu thành một mảng vecto tương ứng mỗi token
        words = word_tokenize(doc.lower())
        words_vector = [model.wv[word] for word in words if word in model.wv]
        if len(words_vector) == 0:
            return np.zeros(model.vector_size)
        return np.mean(words_vector, axis=0)

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
        # Lấy trung bình embeddings của tất cả token
        return np.mean(embeddings.numpy()[0], axis=0)


    if vectorizer == "TfidfVectorizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        vectorizer = TfidfVectorizer(stop_words='english')
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "BagOfWord":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)
        vectorizer = CountVectorizer()
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if  vectorizer == "BagOfWordN_GRAM":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=45)
        vectorizer = CountVectorizer(ngram_range=(2,2))
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "One Hot Encoder":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=45)
        vectorizer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        x_train = np.array(x_train).reshape(-1, 1)
        x_train = vectorizer.fit_transform(x_train)
        x_test = np.array(x_test).reshape(-1, 1)
        x_test = vectorizer.transform(x_test)
        input = np.array(input).reshape(-1, 1)
        input = vectorizer.transform(input)
    if vectorizer == "Word2Vec":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        model = Word2Vec(sentences=x_tokenized, vector_size=100, window=5, min_count=2)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "FastText":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        model = FastText(sentences=x_tokenized, vector_size=100, window=5, min_count=2, min_n=3, max_n=6)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "BertTokenizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        x_train_vector = np.array([get_bert_embedding(doc) for doc in x_train])
        x_test_vector = np.array([get_bert_embedding(doc) for doc in x_test])
        input_vector = np.array([get_bert_embedding(input)])

        # Chuẩn hóa vì MultinomialNB yêu cầu không âm
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train_vector)
        x_test = scaler.transform(x_test_vector)
        input = scaler.transform(input_vector)



    # model = MultinomialNB()
    # model.fit(x_train, y_train)     

    # pred = model.predict(x_test)
    # inputpredect = model.predict(input)

    # metric = metrics.classification_report(y_test, pred, output_dict=True)

    # accuracymodel = metric['accuracy']
    # return accuracymodel, inputpredect[0]
multinomialnb("OneHotEncoder", "fetch_20newsgroups", "I am phan van quan")
 
# acc, result = multinomialnb("One Hot Encoder", "fetch_20newsgroups", "I am phan van quan")
# print(acc)
# print(result)


MemoryError: Unable to allocate 2.53 GiB for an array with shape (9051,) and data type <U75154

### LogisticRegression

In [10]:
from sklearn.linear_model import LogisticRegression

def multinomialnb(vectorizer, data, input):
    if data == "fetch_20newsgroups":
        datasets = fetch_20newsgroups()
    x, y = datasets.data, datasets.target
    input = np.array([input])

    def document_to_vector(doc, model): # chuyển list token trong câu thành một mảng vecto tương ứng mỗi token
        words = word_tokenize(doc.lower())
        words_vector = [model.wv[word] for word in words if word in model.wv]
        if len(words_vector) == 0:
            return np.zeros(model.vector_size)
        return np.mean(words_vector, axis=0)

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
        # Lấy trung bình embeddings của tất cả token
        return np.mean(embeddings.numpy()[0], axis=0)


    if vectorizer == "TfidfVectorizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        vectorizer = TfidfVectorizer(stop_words='english')
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "BagOfWord":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)
        vectorizer = CountVectorizer()
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if  vectorizer == "BagOfWordN_GRAM":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=45)
        vectorizer = CountVectorizer(ngram_range=(2,2))
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "One Hot Encoder":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=45)
        vectorizer = OneHotEncoder()
        x_train = np.array(x_train).reshape(1, -1)
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "Word2Vec":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        model = Word2Vec(sentences=x_tokenized, vector_size=100, window=5, min_count=2)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "FastText":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        model = FastText(sentences=x_tokenized, vector_size=100, window=5, min_count=2, min_n=3, max_n=6)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "BertTokenizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        x_train_vector = np.array([get_bert_embedding(doc) for doc in x_train])
        x_test_vector = np.array([get_bert_embedding(doc) for doc in x_test])
        input_vector = np.array([get_bert_embedding(input)])

        # Chuẩn hóa vì MultinomialNB yêu cầu không âm
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train_vector)
        x_test = scaler.transform(x_test_vector)
        input = scaler.transform(input_vector)



    model = LogisticRegression()
    model.fit(x_train, y_train)     

    pred = model.predict(x_test)
    inputpredect = model.predict(input)

    metric = metrics.classification_report(y_test, pred, output_dict=True)

    accuracymodel = metric['accuracy']
    return accuracymodel, inputpredect[0]

 
acc, result = multinomialnb("TfidfVectorizer", "fetch_20newsgroups", "I am phan van quan")
print(acc)
print(result)

0.8871870397643593
1


## SVM

In [ ]:
from sklearn.svm import SVC

def multinomialnb(vectorizer, data, input):
    if data == "fetch_20newsgroups":
        datasets = fetch_20newsgroups()
    x, y = datasets.data, datasets.target
    input = np.array([input])

    def document_to_vector(doc, model): # chuyển list token trong câu thành một mảng vecto tương ứng mỗi token
        words = word_tokenize(doc.lower())
        words_vector = [model.wv[word] for word in words if word in model.wv]
        if len(words_vector) == 0:
            return np.zeros(model.vector_size)
        return np.mean(words_vector, axis=0)

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
        # Lấy trung bình embeddings của tất cả token
        return np.mean(embeddings.numpy()[0], axis=0)


    if vectorizer == "TfidfVectorizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        vectorizer = TfidfVectorizer(stop_words='english')
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "BagOfWord":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)
        vectorizer = CountVectorizer()
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if  vectorizer == "BagOfWordN_GRAM":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=45)
        vectorizer = CountVectorizer(ngram_range=(2,2))
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "OneHotEncoder":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=45)
        vectorizer = OneHotEncoder()
        x_train = np.array(x_train).reshape(1, -1)
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "Word2Vec":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        model = Word2Vec(sentences=x_tokenized, vector_size=100, window=5, min_count=2)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "FastText":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        model = FastText(sentences=x_tokenized, vector_size=100, window=5, min_count=2, min_n=3, max_n=6)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "BertTokenizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        x_train_vector = np.array([get_bert_embedding(doc) for doc in x_train])
        x_test_vector = np.array([get_bert_embedding(doc) for doc in x_test])
        input_vector = np.array([get_bert_embedding(input)])

        # Chuẩn hóa vì MultinomialNB yêu cầu không âm
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train_vector)
        x_test = scaler.transform(x_test_vector)
        input = scaler.transform(input_vector)



    model = SVC(kernel='rbf', C=1.0, gamma='scale')
    model.fit(x_train, y_train)     

    pred = model.predict(x_test)
    inputpredect = model.predict(input)

    metric = metrics.classification_report(y_test, pred, output_dict=True)

    accuracymodel = metric['accuracy']
    return accuracymodel, inputpredect[0]

 
acc, result = multinomialnb("TfidfVectorizer", "fetch_20newsgroups", "I am phan van quan")
print(acc)
print(result)

### KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


def kneighborsclassifier(vectorizer, data, input):
    if data == "fetch_20newsgroups":
        datasets = fetch_20newsgroups()
    x, y = datasets.data, datasets.target
    input = np.array([input])

    def document_to_vector(doc, model): # chuyển list token trong câu thành một mảng vecto tương ứng mỗi token
        words = word_tokenize(doc.lower())
        words_vector = [model.wv[word] for word in words if word in model.wv]
        if len(words_vector) == 0:
            return np.zeros(model.vector_size)
        return np.mean(words_vector, axis=0)

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
        # Lấy trung bình embeddings của tất cả token
        return np.mean(embeddings.numpy()[0], axis=0)


    if vectorizer == "TfidfVectorizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        vectorizer = TfidfVectorizer(stop_words='english')
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "BagOfWord":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)
        vectorizer = CountVectorizer()
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if  vectorizer == "BagOfWordN_GRAM":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=45)
        vectorizer = CountVectorizer(ngram_range=(2,2))
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "OneHotEncoder":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=45)
        vectorizer = OneHotEncoder()
        x_train = np.array(x_train).reshape(1, -1)
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "Word2Vec":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        model = Word2Vec(sentences=x_tokenized, vector_size=100, window=5, min_count=2)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "FastText":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        model = FastText(sentences=x_tokenized, vector_size=100, window=5, min_count=2, min_n=3, max_n=6)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "BertTokenizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        x_train_vector = np.array([get_bert_embedding(doc) for doc in x_train])
        x_test_vector = np.array([get_bert_embedding(doc) for doc in x_test])
        input_vector = np.array([get_bert_embedding(input)])

        # Chuẩn hóa vì MultinomialNB yêu cầu không âm
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train_vector)
        x_test = scaler.transform(x_test_vector)
        input = scaler.transform(input_vector)



    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(x_train, y_train)     

    pred = model.predict(x_test)
    inputpredect = model.predict(input)

    metric = metrics.classification_report(y_test, pred, output_dict=True)

    accuracymodel = metric['accuracy']
    return accuracymodel, inputpredect[0]

 
acc, result = multinomialnb("TfidfVectorizer", "fetch_20newsgroups", "I am phan van quan")
print(acc)
print(result)

### DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier


def kneighborsclassifier(vectorizer, data, input):
    if data == "fetch_20newsgroups":
        datasets = fetch_20newsgroups()
    x, y = datasets.data, datasets.target
    input = np.array([input])

    def document_to_vector(doc, model): # chuyển list token trong câu thành một mảng vecto tương ứng mỗi token
        words = word_tokenize(doc.lower())
        words_vector = [model.wv[word] for word in words if word in model.wv]
        if len(words_vector) == 0:
            return np.zeros(model.vector_size)
        return np.mean(words_vector, axis=0)

    def get_bert_embedding(text):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            embeddings = outputs.last_hidden_state
        # Lấy trung bình embeddings của tất cả token
        return np.mean(embeddings.numpy()[0], axis=0)


    if vectorizer == "TfidfVectorizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        vectorizer = TfidfVectorizer(stop_words='english')
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "BagOfWord":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)
        vectorizer = CountVectorizer()
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if  vectorizer == "BagOfWordN_GRAM":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=45)
        vectorizer = CountVectorizer(ngram_range=(2,2))
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "OneHotEncoder":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=45)
        vectorizer = OneHotEncoder()
        x_train = np.array(x_train).reshape(1, -1)
        x_train = vectorizer.fit_transform(x_train)
        x_test = vectorizer.transform(x_test)
        input = vectorizer.transform(input)
    if vectorizer == "Word2Vec":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=45)
        model = Word2Vec(sentences=x_tokenized, vector_size=100, window=5, min_count=2)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "FastText":
        x_tokenized = [word_tokenize(doc.lower()) for doc in x] # lấy từng row trong x -> tách ra từng từ -> chuyển chữ thường
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        model = FastText(sentences=x_tokenized, vector_size=100, window=5, min_count=2, min_n=3, max_n=6)

        x_train_vector = np.array([document_to_vector(doc, model) for doc in x_train])
        x_test_vector = np.array([document_to_vector(doc, model) for doc in x_test])
        input_vector = np.array([document_to_vector(input[0], model)])

        scalar = MinMaxScaler()
        x_train = scalar.fit_transform(x_train_vector)
        x_test = scalar.transform(x_test_vector)
        input = scalar.transform(input_vector)
    if vectorizer == "BertTokenizer":
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=45)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        x_train_vector = np.array([get_bert_embedding(doc) for doc in x_train])
        x_test_vector = np.array([get_bert_embedding(doc) for doc in x_test])
        input_vector = np.array([get_bert_embedding(input)])

        # Chuẩn hóa vì MultinomialNB yêu cầu không âm
        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train_vector)
        x_test = scaler.transform(x_test_vector)
        input = scaler.transform(input_vector)



    model = DecisionTreeClassifier(n_neighbors=5)
    model.fit(x_train, y_train)     

    pred = model.predict(x_test)
    inputpredect = model.predict(input)

    metric = metrics.classification_report(y_test, pred, output_dict=True)

    accuracymodel = metric['accuracy']
    return accuracymodel, inputpredect[0]

 
acc, result = multinomialnb("One Hot Encoder", "fetch_20newsgroups", "I am phan van quan")
print(acc)
print(result)